In [1]:
import os, sys
sys.path.append("/home/holywater2/crystal/DeepDFT")
import numpy as np
import torch
import dataset

/home/holywater2/.conda/envs/infGCN/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dd = dataset.DensityDataDir("/home/holywater2/crystal/dataset_mp")

In [6]:
dd[0].keys()

dict_keys(['density', 'atoms', 'origin', 'grid_position', 'metadata'])

In [8]:
dd[0]["density"].shape

(36, 36, 36)

In [11]:
dd[0]["atoms"]

Atoms(symbols='Fe', pbc=True, cell=[[-2.5e-05, -0.000273, -2.459206], [1.159251, -2.008215, 0.819524], [-2.318956, -0.000185, 0.819712]])

In [12]:
dd[0]["origin"]

array([0., 0., 0.])

In [13]:
dd[0]["grid_position"]

array([[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [-6.44154444e-02, -5.13888889e-06,  2.27697778e-02],
         [-1.28830889e-01, -1.02777778e-05,  4.55395556e-02],
         ...,
         [-2.12570967e+00, -1.69583333e-04,  7.51402667e-01],
         [-2.19012511e+00, -1.74722222e-04,  7.74172444e-01],
         [-2.25454056e+00, -1.79861111e-04,  7.96942222e-01]],

        [[ 3.22014167e-02, -5.57837500e-02,  2.27645556e-02],
         [-3.22140278e-02, -5.57888889e-02,  4.55343333e-02],
         [-9.66294722e-02, -5.57940278e-02,  6.83041111e-02],
         ...,
         [-2.09350825e+00, -5.59533333e-02,  7.74167222e-01],
         [-2.15792369e+00, -5.59584722e-02,  7.96937000e-01],
         [-2.22233914e+00, -5.59636111e-02,  8.19706778e-01]],

        [[ 6.44028333e-02, -1.11567500e-01,  4.55291111e-02],
         [-1.26111111e-05, -1.11572639e-01,  6.82988889e-02],
         [-6.44280556e-02, -1.11577778e-01,  9.10686667e-02],
         ...,
         [-2.06130683e+0

In [14]:
dd[0]["metadata"]

{'filename': 'mp-13.chgcar'}

In [15]:
import ase

In [18]:
ase.Atoms()

Atoms(symbols='', pbc=False)

In [20]:
sys.path.append("/home/holywater2/crystal/InfGCN-pytorch/source/datasets")

In [21]:
import density

In [25]:
# root = "/home/holywater2/crystal/dataset_mp"
# split = "train"
# split_file = "test_split.json"
# atom_file = "/home/holywater2/crystal/InfGCN-pytorch/atom_info/mp_cubic_atom_info.json"
# dd2 = density.DensityDataset(root, split, split_file, atom_file,extension="chgcar")

In [29]:
path_dir = "../../../dataset_mp"

saved_files = os.listdir(path_dir)
saved_files_wo_ext = []
for i in saved_files:
    if i.split(".")[1] == "chgcar":
        saved_files_wo_ext.append(i.split(".")[0])

In [30]:
print(i)

mp-7.chgcar


In [78]:
item = 3
file_pattern = ".chgcar"
file_list = saved_files_wo_ext
print(file_list[item])

file_name = f'{(file_list[item])}{file_pattern}'
data_path = path_dir

mp-4


In [70]:
from torch_geometric.data import Data

In [101]:
fileobj = open(os.path.join(data_path, file_name), 'r')
readline = fileobj.readline
readline()  # the first comment line
scale = float(readline())  # the scaling factor (lattice constant)

# the upcoming three lines contain the cell information
cell = torch.empty(3, 3, dtype=torch.float)
for i in range(3):
    cell[i] = torch.FloatTensor([float(s) for s in readline().split()])
cell = cell * scale

# the sixth line specifies the constituting elements
elements = readline().split()
# the seventh line supplies the number of atoms per atomic species
n_atoms = [int(s) for s in readline().split()]
# the eighth line is always "Direct" in our application
readline()

tot_atoms = sum(n_atoms)
atom_type = []
atom_coord = torch.empty(tot_atoms, 3, dtype=torch.float)
# the upcoming lines contains the atomic positions in fractional coordinates
idx = 0
for elem, n in zip(elements, n_atoms):
    atom_type += [elem] * n
    for _ in range(n):
        atom_coord[idx] = torch.FloatTensor(
            [float(s) for s in readline().split()]
        )
        idx += 1

# the coordinates are fractional, convert them to cartesian
atom_coord = atom_coord @ cell
g = Data(x=atom_type, pos=atom_coord)

readline()  # an empty line
shape = [int(s) for s in readline().split()]  # grid size
n_grid = shape[0] * shape[1] * shape[2]
# the grids are corner-aligned
x_coord = (
    torch.linspace(0, shape[0] - 1, shape[0]).unsqueeze(-1) / shape[0] * cell[0]
)
y_coord = (
    torch.linspace(0, shape[1] - 1, shape[1]).unsqueeze(-1) / shape[1] * cell[1]
)
z_coord = (
    torch.linspace(0, shape[2] - 1, shape[2]).unsqueeze(-1) / shape[2] * cell[2]
)
grid_coord = (
    x_coord.view(-1, 1, 1, 3)
    + y_coord.view(1, -1, 1, 3)
    + z_coord.view(1, 1, -1, 3)
)
grid_coord = grid_coord.view(-1, 3)

# the augmented occupancies are ignored
density = torch.FloatTensor([float(s) for s in fileobj.read().split()[:n_grid]])
# the value stored is the charge within a grid instead of the charge density
# divide the charge by the grid volume to get the density
volume = torch.linalg.det(cell).abs()
density = density / volume
# CHGCAR file stores the density as Z-Y-X, convert them to X-Y-Z
density = (
    density.view(shape[2], shape[1], shape[0])
    .transpose(0, 2)
    .contiguous()
    .view(-1)
)

In [72]:
g

Data(x=[2], pos=[2, 3])

In [73]:
density.shape

torch.Size([165888])

In [74]:
from ase.calculators.vasp import VaspChargeDensity

In [79]:
vasp_charge = VaspChargeDensity(filename=os.path.join(data_path, file_name))

In [80]:
vasp_charge.chg[-1]

array([[[3.39629885, 3.37699267, 3.31969071, ..., 3.22616187,
         3.31969071, 3.37699267],
        [3.37699267, 3.370587  , 3.32603846, ..., 3.17122945,
         3.27576902, 3.34505201],
        [3.31969071, 3.32603846, 3.29461406, ..., 3.08153108,
         3.19560867, 3.27576902],
        ...,
        [3.22616187, 3.17122945, 3.08153108, ..., 3.17165224,
         3.22638073, 3.24473147],
        [3.31969071, 3.27576902, 3.19560867, ..., 3.22638073,
         3.29461406, 3.32603846],
        [3.37699267, 3.34505201, 3.27576902, ..., 3.24473147,
         3.32603846, 3.370587  ]],

       [[3.37699267, 3.370587  , 3.32603846, ..., 3.17122945,
         3.27576902, 3.34505201],
        [3.370587  , 3.37699266, 3.34505198, ..., 3.12906497,
         3.2447315 , 3.32603848],
        [3.32603846, 3.34505198, 3.32603844, ..., 3.05213022,
         3.177411  , 3.26959019],
        ...,
        [3.17122945, 3.12906497, 3.05213022, ..., 3.08212992,
         3.14742419, 3.17741099],
        [3.2

In [103]:
vasp_charge.atoms[-1]

Atoms(symbols='Nd', pbc=True, cell=[[2.066667, 2.066667, 2.066667], [2.066667, -2.066667, -2.066667], [-2.066667, 2.066667, -2.066667]])

In [52]:
len(vasp_charge.chg)

1

In [53]:
vasp_charge.chg[-1].shape

(48, 48, 72)

In [59]:
density.view(shape[0], shape[1], shape[2])

tensor([[[0.1952, 0.1960, 0.1984,  ..., 0.2024, 0.1984, 0.1960],
         [0.1959, 0.1967, 0.1991,  ..., 0.2030, 0.1991, 0.1967],
         [0.1977, 0.1986, 0.2010,  ..., 0.2050, 0.2010, 0.1985],
         ...,
         [0.2009, 0.2017, 0.2042,  ..., 0.2083, 0.2042, 0.2017],
         [0.1977, 0.1985, 0.2010,  ..., 0.2050, 0.2010, 0.1985],
         [0.1959, 0.1967, 0.1991,  ..., 0.2031, 0.1991, 0.1967]],

        [[0.1959, 0.1967, 0.1991,  ..., 0.2031, 0.1991, 0.1967],
         [0.1959, 0.1967, 0.1991,  ..., 0.2031, 0.1991, 0.1967],
         [0.1971, 0.1979, 0.2004,  ..., 0.2043, 0.2003, 0.1979],
         ...,
         [0.2034, 0.2043, 0.2069,  ..., 0.2106, 0.2066, 0.2041],
         [0.1996, 0.2005, 0.2030,  ..., 0.2069, 0.2028, 0.2004],
         [0.1971, 0.1979, 0.2004,  ..., 0.2043, 0.2003, 0.1979]],

        [[0.1977, 0.1985, 0.2010,  ..., 0.2050, 0.2010, 0.1985],
         [0.1971, 0.1979, 0.2003,  ..., 0.2044, 0.2004, 0.1979],
         [0.1977, 0.1985, 0.2010,  ..., 0.2050, 0.2010, 0.

In [60]:
atom_type

tensor([139138912, 139138896])

In [87]:
atom_coord

tensor([[9.4759e-10, 4.5616e-41, 1.6507e-33],
        [0.0000e+00, 4.4842e-44, 0.0000e+00]])

In [86]:
elements

['Tb']

In [102]:
atom_type

['Nd']

In [95]:
atoms = ase.Atoms('HHCCHH', [[-1, 1, 0], [-1, -1, 0], [0, 0, 0],
                         [1, 0, 0], [2, 1, 0], [2, -1, 0]])

In [96]:
atoms

Atoms(symbols='H2C2H2', pbc=False)

In [99]:
[] + 'Tb'*2

TypeError: can only concatenate list (not "str") to list

In [104]:
import pandarellel

ModuleNotFoundError: No module named 'pandarellel'

In [105]:
!pip install pandarellel

ERROR: Could not find a version that satisfies the requirement pandarellel (from versions: none)
ERROR: No matching distribution found for pandarellel


In [107]:
df = pd.DataFrame({"a": [1,2,3,4,5,6,7,8,9,10], "b": [1,2,3,4,5,6,7,8,9,10]})

NameError: name 'pd' is not defined

In [108]:
import pandas as pd

In [110]:
df = pd.DataFrame([1,2,3])

In [113]:
df.apply(lambda x: x**2, axis=1)

,0
0,1
1,4
2,9


In [114]:
df

,0
0,1
1,2
2,3


In [115]:
from mpire import WorkerPool

with WorkerPool(n_jobs=5) as pool:
    results = pool.map(lambda x: x+1, range(10))